In [1]:
from pylab import plt
import pandas as pd
import numpy as np
import sympy as sy
import matplotlib.pyplot as plt
import datetime as dt
import scipy as sc
from pylab import mpl
import scipy.integrate as sci
import scipy.stats as scs
import pandas_datareader as web
import statsmodels.api as sm
import enum
import scipy.optimize as spo
from numpy.fft import *
from yahoo_fin import options
from dawp import *
from mystic.solvers import fmin, fmin_powell, diffev, diffev2, sparsity,lattice
import math
import sys
import yfinance as yf

sys.path.append('../')
sys.path.append('C:/Users/kiedy/dx')

import autograd.numpy as agnp
from autograd import grad

np.set_printoptions(suppress=True, formatter={'all': lambda x: '%5.4f' % x})

In [2]:
options1=pd.DataFrame([43.10, 35.60, 22.90, 20.20, 13.30],
                     columns=['price'])
options1['strike']=(1090,1100,1120,1125,1140)
options1['maturity']=dt.datetime(2002,5,17)
options1['T_2']=0.088


options2=pd.DataFrame([84.50, 64.30, 39.50, 33.50, 30.70, 28.00,  25.60, 23.20,19.10,15.30,12.10,10.90],
                     columns=['price'])
options2['strike']=(1050,1075,1110,1120,1125,1130,1135,1140,1150,1160,1170,1175)
options2['maturity']=dt.datetime(2002,6,21)
options2['T_2']=0.184


options3=pd.DataFrame([161.60, 144.80, 120.10, 100.70, 82.50,65.50,51.00,45.50,38.10,27.70,19.60,13.20],
                     columns=['price'])
options3['strike']=(975,995,1025,1050,1075,1100,1125,1135,1150,1175,1200,1225)
options3['maturity']=dt.datetime(2002,9,20)
options3['T_2']=0.436



options4=pd.DataFrame([173.30,157.00,133.10,114.80,97.60,81.20,66.90,58.90,53.90,42.50,33.00,24.90,18.30,13.20],
                     columns=['price'])
options4['strike']=(975,995,1025,1050,1075,1100,1125,1140,1150,1175,1200,1225,1250,1275)
options4['maturity']=dt.datetime(2002,12,20)
options4['T_2']=0.692


options5=pd.DataFrame([146.50,96.20,81.70,68.30,56.60,46.10,36.90,29.30,22.50,17.20,12.80],
                     columns=['price'])
options5['strike']=(1025,1100,1125,1150,1175,1200,1225,1250,1275,1300,1325)
options5['maturity']=dt.datetime(2003,3,21)
options5['T_2']=0.936


options6=pd.DataFrame([182.10,143.00,111.30,97.00,83.30,60.90,49.80,41.20,27.10,17.10,10.10],
                     columns=['price'])
options6['strike']=(995,1050,1100,1125,1150,1200,1225,1250,1300,1350,1400)
options6['maturity']=dt.datetime(2003,6,20)
options6['T_2']=1.192

options7=pd.DataFrame([171.40,140.40,112.80,99.80,66.90,49.50,35.70,25.20,17.00,12.20],
                     columns=['price'])
options7['strike']=(1050,1100,1150,1175,1250,1300,1350,1400,1450,1500)
options7['maturity']=dt.datetime(2003,12,19)
options7['T_2']=1.708

frames=[options1, options2, options3, options4, options5, options6, options7]

options=pd.concat(frames, ignore_index=True)
options['pricing_date'] = dt.datetime(2002, 4, 18)
options['T']=(options['maturity']-options['pricing_date']).dt.days/365.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


r=0.019
q=0.012
S0=1124.47
i = np.complex(0.0, 1.0)

# Metoda COS

In [3]:
def cf(u, T, r, theta, beta, sigma):
    ''' Funkcja charakterystyczna w modelu CGMY '''
    b = r - q +(1/beta)*np.log(1-theta*beta-0.5*sigma**2*beta)
    value = np.exp(i * u * b * T)*(1-i*u*theta*beta+0.5*sigma**2*beta*u**2)**(-T/beta)
    return value

In [4]:
# def G(u, *args):
#     ''' Funkcja generująca kumulanty'''
#     return np.log(cf(-1j*u,T,r, theta, beta, sigma))

# T=1
# theta=0
# beta=0.1
# sigma=0.2

In [5]:
N=2**12
L=10
h=1e-3
    
def CallPutOptionPriceCOSMthd(cf,S0,r,T,K,theta, beta, sigma,N,L,CALL):


    # cf   - Characteristic function is a function, in the book denoted by \varphi
    # CP   - C for call and P for put
    # S0   - Initial stock price
    # r    - Interest rate (constant)
    # T    - Time to maturity
    # K    - List of strikes
    # N    - Number of expansion terms
    # L    - Size of truncation domain (typ.:L=8 or L=10)
        
    # Reshape K to become a column vector

    if K is not np.array:
        K = np.array(K).reshape([len(K),1])
        
    x0 = np.log(S0 / K)
    
    # kumulanty
    w=(1/beta)*np.log(1-theta*beta-0.5*sigma**2*beta)
    c1=np.log(S0/K)+T*(r-q-w+theta)
    c2=(sigma**2+beta*theta**2)*T
    c4=3*T*(sigma**4*beta+2*theta**4*beta**3+4*sigma**2*theta**2*beta**2)
    
    # kumulanty z automatycznego różniczkowania
#     c1=grad_g(0.)
#     c2=grad_g2(0.)
#     c4=grad_g4(0.)

    # kumulanty z metody różnic skończonych
#     c1=(G(h)-G(-h))/(2*h)
#     c2=(G(h)-2*G(0)+G(-h))/(h**2)
#     c4=(G(3*h)-2*G(2*h)+4*G(0)-G(h)-G(-h)-2*G(-2*h)+G(-3*h))/(4*h**4)

    a=c1+x0-L*np.sqrt(c2+np.sqrt(c4))
    b=c1+x0+L*np.sqrt(c2+np.sqrt(c4))
    # Summation from k = 0 to k=N-1

    k = np.linspace(0,N-1,N).reshape([N,1])  
    u = k * np.pi / (b - a);  

    # Determine coefficients for put prices  

    H_k = CallPutCoefficients(a,b,k)   
    mat = np.exp(1j * np.outer((x0 - a) , u))
    temp = cf(u, T,r,theta, beta, sigma) * H_k 
    temp[0] = 0.5 * temp[0]    
    value = np.exp(-r * T) * K * np.real(mat.dot(temp))
    if CALL == True:
        return  value + np.exp(-q*T)*S0 - K*np.exp(-r*T)
    else:
        return value

# Determine coefficients for put prices 

def CallPutCoefficients(a,b,k):
    
    c = a
    d = 0.0
    coef = Chi_Psi(a,b,c,d,k)
    Chi_k = coef["chi"]
    Psi_k = coef["psi"]
    if a<b and b<0.0:
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k) 
    elif a>0 and a<b:
        H_k = np.zeros([len(k),1])
    else:
        H_k = 2.0 / (b - a) * (- Chi_k + Psi_k)
    
    return H_k    

def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value

In [7]:
# Projekt MNSDE

theta = - 0.1370
nu = 0.7206
sigma = 0.1791
K = 1100
T = 1
CallPutOptionPriceCOSMthd(cf,S0,r,T,(K,),theta, nu, sigma,N,L,True)

array([[101.2161]])

In [8]:
m=0
def VG_error_function(p0):
    
    global m
    theta, beta, sigma=p0
    se = []
    for row, option in options.iterrows():
        model_value = CallPutOptionPriceCOSMthd(cf,S0,r,option['T_2'],(option['strike'],),theta, beta, sigma,N,L,True)[0][0]
        if np.isnan(model_value):
            model_value=np.inf
        se.append((model_value - option['price']) ** 2)
    RMSE = (sum(se) / len(se))**0.5
    if m % 1 == 0:
        print('%4d |' % m, np.array(p0), '| %7.3f' % RMSE)
        
    m += 1
    return RMSE

VG_error_function((-0.1370,0.7206,0.1791))

   0 | [-0.1370 0.7206 0.1791] |   3.557


3.5574073846540593

In [9]:
def VG_calibration_full():
    i=0
    global opt
    p0 = spo.brute(VG_error_function, ((-0.5,0,0.05),(0.001,1, 0.1), (0.001,0.41, 0.05)), finish=None)
    opt = spo.fmin(VG_error_function, p0, xtol=0.000001, ftol=0.000001, maxiter=15000, maxfun=15000)
    return opt

VG_calibration_full()

   1 | [-0.5000 0.0010 0.0010] | 514.714
   2 | [-0.5000 0.0010 0.0510] | 461.125
   3 | [-0.5000 0.0010 0.1010] | 118.715
   4 | [-0.5000 0.0010 0.1510] |  11.850
   5 | [-0.5000 0.0010 0.2010] |   9.908
   6 | [-0.5000 0.0010 0.2510] |  26.104
   7 | [-0.5000 0.0010 0.3010] |  44.159
   8 | [-0.5000 0.0010 0.3510] |  62.692
   9 | [-0.5000 0.0010 0.4010] |  81.434
  10 | [-0.5000 0.1010 0.0010] |  11.366
  11 | [-0.5000 0.1010 0.0510] |   9.174
  12 | [-0.5000 0.1010 0.1010] |   4.881
  13 | [-0.5000 0.1010 0.1510] |  10.458
  14 | [-0.5000 0.1010 0.2010] |  22.506
  15 | [-0.5000 0.1010 0.2510] |  36.625
  16 | [-0.5000 0.1010 0.3010] |  51.944
  17 | [-0.5000 0.1010 0.3510] |  68.046
  18 | [-0.5000 0.1010 0.4010] |  84.679
  19 | [-0.5000 0.2010 0.0010] |   9.130
  20 | [-0.5000 0.2010 0.0510] |  10.760
  21 | [-0.5000 0.2010 0.1010] |  15.579
  22 | [-0.5000 0.2010 0.1510] |  23.332
  23 | [-0.5000 0.2010 0.2010] |  33.441
  24 | [-0.5000 0.2010 0.2510] |  45.324
  25 | [-0.5000 

 201 | [-0.4000 0.2010 0.1010] |   4.896
 202 | [-0.4000 0.2010 0.1510] |  12.522
 203 | [-0.4000 0.2010 0.2010] |  23.805
 204 | [-0.4000 0.2010 0.2510] |  36.973
 205 | [-0.4000 0.2010 0.3010] |  51.374
 206 | [-0.4000 0.2010 0.3510] |  66.628
 207 | [-0.4000 0.2010 0.4010] |  82.492
 208 | [-0.4000 0.3010 0.0010] |   6.243
 209 | [-0.4000 0.3010 0.0510] |   7.679
 210 | [-0.4000 0.3010 0.1010] |  12.229
 211 | [-0.4000 0.3010 0.1510] |  19.783
 212 | [-0.4000 0.3010 0.2010] |  29.710
 213 | [-0.4000 0.3010 0.2510] |  41.398
 214 | [-0.4000 0.3010 0.3010] |  54.379
 215 | [-0.4000 0.3010 0.3510] |  68.320
 216 | [-0.4000 0.3010 0.4010] |  82.989
 217 | [-0.4000 0.4010 0.0010] |  13.683
 218 | [-0.4000 0.4010 0.0510] |  15.119
 219 | [-0.4000 0.4010 0.1010] |  19.273
 220 | [-0.4000 0.4010 0.1510] |  25.936
 221 | [-0.4000 0.4010 0.2010] |  34.722
 222 | [-0.4000 0.4010 0.2510] |  45.198
 223 | [-0.4000 0.4010 0.3010] |  56.986
 224 | [-0.4000 0.4010 0.3510] |  69.793
 225 | [-0.4000 

 403 | [-0.3000 0.4010 0.3010] |  47.323
 404 | [-0.3000 0.4010 0.3510] |  61.733
 405 | [-0.3000 0.4010 0.4010] |  76.829
 406 | [-0.3000 0.5010 0.0010] |   4.312
 407 | [-0.3000 0.5010 0.0510] |   4.537
 408 | [-0.3000 0.5010 0.1010] |   7.594
 409 | [-0.3000 0.5010 0.1510] |  14.497
 410 | [-0.3000 0.5010 0.2010] |  24.024
 411 | [-0.3000 0.5010 0.2510] |  35.353
 412 | [-0.3000 0.5010 0.3010] |  47.976
 413 | [-0.3000 0.5010 0.3510] |  61.568
 414 | [-0.3000 0.5010 0.4010] |  75.915
 415 | [-0.3000 0.6010 0.0010] |   6.131
 416 | [-0.3000 0.6010 0.0510] |   7.223
 417 | [-0.3000 0.6010 0.1010] |  10.921
 418 | [-0.3000 0.6010 0.1510] |  17.409
 419 | [-0.3000 0.6010 0.2010] |  26.191
 420 | [-0.3000 0.6010 0.2510] |  36.714
 421 | [-0.3000 0.6010 0.3010] |  48.547
 422 | [-0.3000 0.6010 0.3510] |  61.390
 423 | [-0.3000 0.6010 0.4010] |  75.043
 424 | [-0.3000 0.7010 0.0010] |   9.181
 425 | [-0.3000 0.7010 0.0510] |  10.367
 426 | [-0.3000 0.7010 0.1010] |  13.963
 427 | [-0.3000 

 606 | [-0.2000 0.7010 0.1010] |   8.139
 607 | [-0.2000 0.7010 0.1510] |   4.546
 608 | [-0.2000 0.7010 0.2010] |  13.161
 609 | [-0.2000 0.7010 0.2510] |  25.241
 610 | [-0.2000 0.7010 0.3010] |  38.678
 611 | [-0.2000 0.7010 0.3510] |  53.041
 612 | [-0.2000 0.7010 0.4010] |  68.129
 613 | [-0.2000 0.8010 0.0010] |  13.103
 614 | [-0.2000 0.8010 0.0510] |  11.485
 615 | [-0.2000 0.8010 0.1010] |   7.129
 616 | [-0.2000 0.8010 0.1510] |   5.341
 617 | [-0.2000 0.8010 0.2010] |  13.715
 618 | [-0.2000 0.8010 0.2510] |  25.212
 619 | [-0.2000 0.8010 0.3010] |  38.115
 620 | [-0.2000 0.8010 0.3510] |  51.997
 621 | [-0.2000 0.8010 0.4010] |  66.657
 622 | [-0.2000 0.9010 0.0010] |  11.559
 623 | [-0.2000 0.9010 0.0510] |  10.136
 624 | [-0.2000 0.9010 0.1010] |   6.487
 625 | [-0.2000 0.9010 0.1510] |   6.194
 626 | [-0.2000 0.9010 0.2010] |  14.231
 627 | [-0.2000 0.9010 0.2510] |  25.179
 628 | [-0.2000 0.9010 0.3010] |  37.577
 629 | [-0.2000 0.9010 0.3510] |  51.001
 630 | [-0.2000 

 810 | [-0.1000 0.9010 0.4010] |  64.607
 811 | [-0.0500 0.0010 0.0010] |  45.149
 812 | [-0.0500 0.0010 0.0510] |  38.570
 813 | [-0.0500 0.0010 0.1010] |  26.409
 814 | [-0.0500 0.0010 0.1510] |  12.066
 815 | [-0.0500 0.0010 0.2010] |   9.792
 816 | [-0.0500 0.0010 0.2510] |  25.976
 817 | [-0.0500 0.0010 0.3010] |  44.057
 818 | [-0.0500 0.0010 0.3510] |  62.615
 819 | [-0.0500 0.0010 0.4010] |  81.377
 820 | [-0.0500 0.1010 0.0010] |  43.958
 821 | [-0.0500 0.1010 0.0510] |  38.245
 822 | [-0.0500 0.1010 0.1010] |  26.467
 823 | [-0.0500 0.1010 0.1510] |  12.334
 824 | [-0.0500 0.1010 0.2010] |   8.788
 825 | [-0.0500 0.1010 0.2510] |  24.570
 826 | [-0.0500 0.1010 0.3010] |  42.361
 827 | [-0.0500 0.1010 0.3510] |  60.670
 828 | [-0.0500 0.1010 0.4010] |  79.234
 829 | [-0.0500 0.2010 0.0010] |  43.306
 830 | [-0.0500 0.2010 0.0510] |  37.938
 831 | [-0.0500 0.2010 0.1010] |  26.524
 832 | [-0.0500 0.2010 0.1510] |  12.636
 833 | [-0.0500 0.2010 0.2010] |   7.980
 834 | [-0.0500 

1013 | [-0.1371 0.7184 0.1791] |   3.557
1014 | [-0.1370 0.7212 0.1791] |   3.557
1015 | [-0.1366 0.7236 0.1793] |   3.557
1016 | [-0.1371 0.7199 0.1791] |   3.557
1017 | [-0.1371 0.7196 0.1791] |   3.557
1018 | [-0.1371 0.7201 0.1791] |   3.557
1019 | [-0.1371 0.7202 0.1791] |   3.557
1020 | [-0.1370 0.7205 0.1791] |   3.557
1021 | [-0.1369 0.7213 0.1791] |   3.557
1022 | [-0.1370 0.7203 0.1791] |   3.557
1023 | [-0.1371 0.7194 0.1791] |   3.557
1024 | [-0.1370 0.7208 0.1791] |   3.557
1025 | [-0.1370 0.7209 0.1791] |   3.557
1026 | [-0.1370 0.7207 0.1791] |   3.557
1027 | [-0.1370 0.7207 0.1791] |   3.557
1028 | [-0.1370 0.7206 0.1791] |   3.557
1029 | [-0.1369 0.7211 0.1791] |   3.557
1030 | [-0.1370 0.7205 0.1791] |   3.557
1031 | [-0.1370 0.7205 0.1791] |   3.557
1032 | [-0.1370 0.7203 0.1791] |   3.557
1033 | [-0.1370 0.7206 0.1791] |   3.557
1034 | [-0.1370 0.7207 0.1791] |   3.557
1035 | [-0.1370 0.7205 0.1791] |   3.557
1036 | [-0.1370 0.7206 0.1791] |   3.557
1037 | [-0.1370 

array([-0.1370, 0.7206, 0.1791])

# Kalibracja dla wielu K

In [7]:
N=2**12
L=10
h=1e-3
    
def CallPutOptionPriceCOSMthd(cf,S0,r,T,K,theta, beta, sigma,N,L,CALL):


    # cf   - Characteristic function is a function, in the book denoted by \varphi
    # CP   - C for call and P for put
    # S0   - Initial stock price
    # r    - Interest rate (constant)
    # T    - Time to maturity
    # K    - List of strikes
    # N    - Number of expansion terms
    # L    - Size of truncation domain (typ.:L=8 or L=10)
        
    # Reshape K to become a column vector

    if K is not np.array:
        K = np.array(K).reshape([len(K),1])
        
    x0 = np.log(S0 / K)
    
    # kumulanty
    w=(1/beta)*np.log(1-theta*beta-0.5*sigma**2*beta)
    c1=T*(r-q-w+theta)
    c2=(sigma**2+beta*theta**2)*T
    c4=3*T*(sigma**4*beta+2*theta**4*beta**3+4*sigma**2*theta**2*beta**2)
    
    # kumulanty z automatycznego różniczkowania
#     c1=grad_g(0.)
#     c2=grad_g2(0.)
#     c4=grad_g4(0.)

    # kumulanty z metody różnic skończonych
#     c1=(G(h)-G(-h))/(2*h)
#     c2=(G(h)-2*G(0)+G(-h))/(h**2)
#     c4=(G(3*h)-2*G(2*h)+4*G(0)-G(h)-G(-h)-2*G(-2*h)+G(-3*h))/(4*h**4)

    a=c1-L*np.sqrt(c2+np.sqrt(c4))
    b=c1+L*np.sqrt(c2+np.sqrt(c4))
    # Summation from k = 0 to k=N-1

    k = np.linspace(0,N-1,N).reshape([N,1])  
    u = k * np.pi / (b - a);  

    # Determine coefficients for put prices  

    H_k = CallPutCoefficients(a,b,k)   
    mat = np.exp(1j * np.outer((x0 - a) , u))
    temp = cf(u, T,r,theta, beta, sigma) * H_k 
    temp[0] = 0.5 * temp[0]    
    value = np.exp(-r * T) * K * np.real(mat.dot(temp))
    if CALL == True:
        return  value + np.exp(-q*T)*S0 - K*np.exp(-r*T)
    else:
        return value

# Determine coefficients for put prices 

def CallPutCoefficients(a,b,k):
    
    c = a
    d = 0.0
    coef = Chi_Psi(a,b,c,d,k)
    Chi_k = coef["chi"]
    Psi_k = coef["psi"]
    if a<b and b<0.0:
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k) 
    elif a>0 and a<b:
        H_k = np.zeros([len(k),1])
    else:
        H_k = 2.0 / (b - a) * (- Chi_k + Psi_k)
    
    return H_k    

def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value

In [8]:
m=0
def VG_error_function(p0):
    
    global m
    theta, beta, sigma = p0
#     if sigma<=0:
#         return 100
    se1 = []
    K = np.array(options1['strike'])
    T=options['T_2'][0]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,theta,beta, sigma, N,L, True)
    i=0
    for row, option in options1.iterrows():
        se1.append((model_value[i] - option['price']) ** 2)
        i+=1
        
        
    se2 = []
    K = np.array(options2['strike'])
    T=options['T_2'][10]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,theta,beta, sigma,N,L, True)
    i=0
    for row, option in options2.iterrows():
        se2.append((model_value[i] - option['price']) ** 2)
        i+=1
        
        
    se3 = []    
    K = np.array(options3['strike'])
    T=options['T_2'][18]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,theta,beta, sigma,N,L, True)
    i=0
    for row, option in options3.iterrows():
        se3.append((model_value[i] - option['price']) ** 2)
        i+=1        
        
        
    se4 = []    
    K = np.array(options4['strike'])
    T=options['T_2'][37]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,theta,beta, sigma,N,L, True)
    i=0
    for row, option in options4.iterrows():
        se4.append((model_value[i] - option['price']) ** 2)
        i+=1        

    se5 = []    
    K = np.array(options5['strike'])
    T=options['T_2'][48]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,theta,beta, sigma,N,L, True)
    i=0
    for row, option in options5.iterrows():
        se5.append((model_value[i] - option['price']) ** 2)
        i+=1 
        
    se6 = []    
    K = np.array(options6['strike'])
    T=options['T_2'][58]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,theta,beta, sigma,N,L, True)
    i=0
    for row, option in options6.iterrows():
        se6.append((model_value[i] - option['price']) ** 2)
        i+=1
        
        
    se7 = []    
    K = np.array(options7['strike'])
    T=options['T_2'][69]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,theta,beta, sigma,N,L, True)
    i=0
    for row, option in options7.iterrows():
        se7.append((model_value[i] - option['price']) ** 2)
        i+=1
        
    RMSE = ((sum(se1)+sum(se2)+sum(se3)+sum(se4)+sum(se5)+sum(se6)+sum(se7)) / (len(se1)+len(se2)+len(se3)+len(se4)+len(se5)+len(se6)+len(se7)))**0.5
    if m % 1 == 0:
        print('%4d |' % m, np.array(p0), '| %7.3f' % RMSE)
        
    m += 1
    return RMSE

# def BS_calibration_full():
#     i=0
#     global opt
#     p0 = spo.brute(BS_error_function, ((0.07,0.4,0.1),), finish=None)
#     opt = spo.fmin(BS_error_function, p0, xtol=0.000001, ftol=0.000001, maxiter=None, maxfun=None)
#     return opt

VG_error_function((-0.1370,0.7206,0.1791))


   0 | [-0.1370 0.7206 0.1791] |   3.557


array([3.5574])